In [1]:
pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show dlt

Name: dlt
Version: 1.13.0
Summary: dlt is an open-source python-first scalable data loading library that does not require any backend to run.
Home-page: https://github.com/dlt-hub
Author: 
Author-email: "dltHub Inc." <services@dlthub.com>
License-Expression: Apache-2.0
Location: /usr/local/python/3.12.1/lib/python3.12/site-packages
Requires: click, fsspec, gitpython, giturlparse, hexbytes, humanize, jsonpath-ng, orjson, packaging, pathvalidate, pendulum, pluggy, pytz, pyyaml, requests, requirements-parser, rich-argparse, semver, setuptools, simplejson, sqlglot, tenacity, tomlkit, typing-extensions, tzdata
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [9]:
import dlt
import requests
import pandas as pd
from datetime import datetime

In [6]:
@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [7]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [10]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-07-11 18:08:52.227638+00:00 and COMPLETED in 6.04 seconds with 4 steps.
Step extract COMPLETED in 0.44 seconds.

Load package 1752257332.74722 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.08 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1752257332.74722 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 5.00 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.99 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/LLM_ZoomCamp/db.qdrant location to store data
Load package 1752257332.74722 is LOADED and contains no failed jobs

Step run COMPLETED in 6.04 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.99 seconds
1 load package(s) were loaded to destination qdrant and i